In [31]:
%load_ext autoreload
%autoreload 2
from replicaEVSE import datautils as du
import os
import dask.dataframe as dd
import pandas as pd

pd.set_option('display.max_columns', 500)

datadir = '../../data/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=8)  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default
client

/Users/matthew.wilde/mambaforge/envs/py311/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60961 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:60961/status,
Dashboard: http://127.0.0.1:60961/status,Workers: 8
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60962,Workers: 8
Dashboard: http://127.0.0.1:60961/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:60994,Total threads: 1
Dashboard: http://127.0.0.1:60996/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:60965,


### load in the trips and pop data and the join of the two for desitinations that end in WA

In [7]:
pop_ddf = dd.read_parquet(datadir+'/northwest_2021_Q4_population.parquet')
trip_thu_ddf = dd.read_parquet(datadir+'/northwest_2021_Q4_thursday_trip.parquet')
merged_ddf = dd.read_parquet(datadir+'/wa_pop_and_trips_sorted.parquet') # len = 51727268

In [47]:
merged_ddf.head(20)

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type
0,18336634547462657450,10000002909730821040,PRIVATE_AUTO,SOCIAL,HOME,0 days 08:20:42,0 days 08:39:00,2.897196,nan,530250111002,47.098457,-119.319023,530250111003,47.117839,-119.303234,residential,single_family,residential,single_family,residential,single_family,residential,single_family,47.085370,-119.319090,47.118737,-119.298334,thursday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213,NaN,NaN,worked_in_person,PUBLIC
1,1694659094182985471,10000002909730821040,PRIVATE_AUTO,HOME,SOCIAL,0 days 09:12:45,0 days 09:19:56,2.867602,nan,530250111003,47.117839,-119.303234,530250111002,47.098457,-119.319023,residential,single_family,residential,single_family,residential,single_family,residential,single_family,47.118737,-119.298334,47.085370,-119.319090,thursday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213,NaN,NaN,worked_in_person,HOME
2,7721623352602932778,10000002909730821040,ON_DEMAND_AUTO,SHOP,HOME,0 days 09:30:08,0 days 09:49:04,8.929170,nan,530250111002,47.098457,-119.319023,530250109022,47.152531,-119.293494,residential,single_family,residential,single_family,commercial,retail,commercial,retail,47.085370,-119.319090,47.144694,-119.278788,thursday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213,NaN,NaN,worked_in_person,PUBLIC
3,3697919619727259534,10000002909730821040,CARPOOL,SHOP,SHOP,0 days 09:54:21,0 days 09:57:14,0.996870,nan,530250109022,47.152531,-119.293494,530250111001,47.123046,-119.285771,commercial,retail,commercial,retail,commercial,retail,commercial,retail,47.144694,-119.278788,47.131541,-119.276257,thursday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213,NaN,NaN,worked_in_person,PUBLIC
4,9977271661694103189,10000002909730821040,WALKING,SHOP,SHOP,0 days 10:23:28,0 days 10:25:01,0.080298,nan,530250111001,47.123046,-119.285771,530250110002,47.125725,-119.262745,commercial,retail,commercial,retail,commercial,retail,

### Ubers

They have population data associated with them. Whew. 

In [10]:
ubers = trip_thu_ddf.loc[trip_thu_ddf['mode'] == 'ON_DEMAND_AUTO']
ubers['person_id'].value_counts().head(3)

990158709039785955      16
9183962885016971228     15
7236881691111520416     14
17416661576243475932    11
14826177188316444853    11
                        ..
16069901924244521885     1
16069935096932885818     1
16069955311810168656     1
1607009112529347857      1
9999937766891715451      1
Name: person_id, Length: 310740, dtype: int64

In [44]:
ubers_with_lots_of_trips = ['990158709039785955', '9183962885016971228', '7236881691111520416']

most_uber_persons_pop = pop_ddf.loc[pop_ddf['person_id'].isin(ubers_with_lots_of_trips)].compute()
most_uber_persons_trip = trip_thu_ddf.loc[trip_thu_ddf['person_id'].isin(ubers_with_lots_of_trips)].sort_values(['person_id', 'start_time']).compute()

In [37]:
most_uber_persons_pop

,household_id,person_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh
151279,1257911991915080230,9183962885016971228,530459604002,530350903003,nan,53045960400,53035090300,nan,25_34,30,M,american_indian_alaska_native,not_hispanic_or_latino,20000_40000,24783.0,employed,high_school,not_attending_school,naics71,head_of_household,0,nan,carpool,owner,same_house,2_person,40000_75000,45875.0,nonfamily_single,1,single_family,core,english,47.418487,-122.839858,47.694072,-122.735271,NaN,NaN,worked_in_person
20020,178612002277391146,7236881691111520416,530299703004,nan,nan,53029970300,nan,nan,35_64,61,F,white,not_hispanic_or_latino,20000_40000,22649.0,notinlf,some_college,not_attending_school,not_working,head_of_household,0,nan,not_working,owner,same_house,2_person,75000_125000,102084.0,married_couple,2,single_family,core,english,48.310406,-122.546254,NaN,NaN,NaN,NaN,under_16_or_notinlf
657,10751185737545128198,990158709039785955,300630002023,300630002022,nan,30063000202,30063000202,nan,35_64,50,F,white,not_hispanic_or_latino,40000_80000,65480.0,employed,bachelors_degree,not_attending_school,naics71,head_of_household,0,nan,carpool,owner,same_house,4_person,75000_125000,108418.0,married_couple,3_plus,single_family,core,english,46.917304,-114.057524,46.887585,-114.065040,NaN,NaN,worked_from_home


In [45]:
most_uber_persons_trip

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng
34203,7736995502836517179,7236881691111520416,ON_DEMAND_AUTO,RECREATION,HOME,0 days 06:05:27,0 days 07:20:15,8.427841,nan,530299703004,48.315647,-122.539777,530299702001,48.339357,-122.665272,residential,single_family,residential,single_family,mixed_use,retail,commercial,non_retail_attraction,48.310406,-122.546254,48.345785,-122.676192
7968,2648301264427475596,7236881691111520416,ON_DEMAND_AUTO,SHOP,RECREATION,0 days 07:45:54,0 days 07:49:39,1.016913,nan,530299702001,48.339357,-122.665272,530299702001,48.339357,-122.665272,mixed_use,retail,commercial,non_retail_attraction,mixed_use,civic_institutional,commercial,retail,48.345785,-122.676192,48.337486,-122.671883
125180,17070552126432805029,7236881691111520416,CARPOOL,SHOP,SHOP,0 days 08:56:38,0 days 09:08:51,1.680499,nan,530299702001,48.339357,-122.665272,530299704001,48.317242,-122.642992,mixed_use,civic_institutional,commercial,retail,commercial,retail,commercial,retail,48.337486,-122.671883,48.324973,-122.649495
107923,17903347197027184601,7236881691111520416,ON_DEMAND_AUTO,SHOP,SHOP,0 days 09:23:21,0 days 09:32:24,1.606964,nan,530299704001,48.317242,-122.642992,530299702001,48.339357,-122.665272,commercial,retail,commercial,retail,mixed_use,civic_institutional,commercial,retail,48.324973,-122.649495,48.337677,-122.670138
56328,22331895165612259,7236881691111520416,WALKING,SHOP,SHOP,0 days 09:42:41,0 days 09:47:51,0.267760,nan,530299702001,48.339357,-122.665272,530299702001,48.339357,-122.665272,mixed_use,civic_institutional,commercial,retail,mixed_use,civic_institutional,commercial,retail,48.337677,-122.670138,48.337486,-122.671883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150545,11080199617177186478,990158709039785955,ON_DEMAND_AUTO,SHOP,EAT,0 days 21:05:02,0 days 21:15:42,1.504734,nan,300630003002,46.873330,-113.997041,300630005002,46.860842,-113.985110,commercial,retail,commercial,retail,civic_institutional,education,commercial,retail,46.874936,-113.992625,46.864118,-113.981714
49284,3896505468762257919,990158709039785955,ON_DEMAND_AUTO,SOCIAL,SHOP,0 days 21:35:56,0 days 21:47:19,3.272424,nan,300630005002,46.860842,-113.985110,300630004001,46.872417,-113.938603,civic_institutional,education,commercial,retail,residential,single_family,residential,single_family,46.864118,-113.981714,46.873373,-113.942431
129577,2341369042023348668,990158709039785955,ON_DEMAND_AUTO,SHOP,SOCIAL,0 days 21:56:58,0 days 22:13:11,3.605401,nan,300630004001,46.872417,-113.938603,300630005002,46.860842,-113.985110,residential,single_family,residential,single_family,civic_institutional,education,commercial,retail,46.873373,-113.942431,46.864118,-113.981714
22974,11869400707093167900,990158709039785955,ON_DEMAND_AUTO,SHOP,SHOP,0 days 22:20:16,0 days 22:34:45,3.195920,nan,300630005002,46.860842,-113.985110,300630010002,46.843680,-114.032508,civic_institutional,education,commercial,retail,commercial,retail,commercial,retail,46.864118,-113.981714,46.848519,-114.034779


### Trucks
only thursday for now

In [13]:
trucks = trip_thu_ddf.loc[trip_thu_ddf['mode'] == 'COMMERCIAL'] # len = 2190600
trucks['person_id'].value_counts().compute()

10000011262318393978    1
4466000335956290558     1
446616179920110017      1
4466161462016108901     1
4466155850483027306     1
                       ..
15536171076438081648    1
15536163747924793994    1
15536162389987112307    1
155361606456951021      1
9999993057755165125     1
Name: person_id, Length: 2190600, dtype: int64

There doesn't seem to be any persons associated with commercial in the population data

Confirmed: No persons in trucking in the population dataset

In [26]:
truck_person_list = list(trucks['person_id'])
pop_ddf = dd.read_parquet(datadir+'/northwest_2021_Q4_population.parquet')
out = pop_ddf[pop_ddf['person_id'].isin(truck_person_list)]
# len(out) = 0
